#### 读取数据

In [1]:
import pandas as pd

In [2]:
fpath = "C:/Users/pxpxz_ct9p1p3/Downloads/Fortune_1000_Companies_by_Revenue.csv"
df = pd.read_csv(fpath)

In [3]:
df.head()

,rank,name,revenues,revenue_percent_change,profits,profits_percent_change,assets,market_value,change_in_rank,employees
0,1,Walmart,"$572,754",2.40%,"$13,673",1.20%,"$244,860","$409,795",-,"2,300,000"
1,2,Amazon,"$469,822",21.70%,"$33,364",56.40%,"$420,549","$1,658,807.30",-,"1,608,000"
2,3,Apple,"$365,817",33.30%,"$94,680",64.90%,"$351,002","$2,849,537.60",-,"154,000"
3,4,CVS Health,"$292,111",8.70%,"$7,910",10.20%,"$232,999","$132,839.20",-,"258,000"
4,5,UnitedHealth Group,"$287,597",11.80%,"$17,285",12.20%,"$212,206","$479,830.30",-,"350,000"


In [4]:
# 替换掉前缀$等
df.loc[:, 'revenues'] = df['revenues'].str.replace('$', '').str.replace(',','').astype('float')
# 替换掉前缀$等
df.loc[:, 'profits'] = df['profits'].str.replace('$', '').str.replace(',','').str.replace('(','').str.replace(')','').str.replace('(','').str.replace('-','0').astype('float')

C:\Users\PXPXZ_~1\AppData\Local\Temp/ipykernel_9700/3712398985.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'revenues'] = df['revenues'].str.replace('$', '').str.replace(',','').astype('float')
C:\Users\PXPXZ_~1\AppData\Local\Temp/ipykernel_9700/3712398985.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'profits'] = df['profits'].str.replace('$', '').str.replace(',','').str.replace('(','').str.replace(')','').str.replace('(','').str.replace('-','0').astype('float')


In [5]:
df.head()

,rank,name,revenues,revenue_percent_change,profits,profits_percent_change,assets,market_value,change_in_rank,employees
0,1,Walmart,572754.0,2.40%,13673.0,1.20%,"$244,860","$409,795",-,"2,300,000"
1,2,Amazon,469822.0,21.70%,33364.0,56.40%,"$420,549","$1,658,807.30",-,"1,608,000"
2,3,Apple,365817.0,33.30%,94680.0,64.90%,"$351,002","$2,849,537.60",-,"154,000"
3,4,CVS Health,292111.0,8.70%,7910.0,10.20%,"$232,999","$132,839.20",-,"258,000"
4,5,UnitedHealth Group,287597.0,11.80%,17285.0,12.20%,"$212,206","$479,830.30",-,"350,000"


In [12]:
# 只选出rank以5开头的的数据用于分析 （没有实际意义，仅供学习此方法而已）
condition = df['rank'].astype('str').str.startswith('5')

In [13]:
# 设置expense=revenue-profit
df[condition]['expenses']=df['revenues']-df['profits']
# 利用condition选出rank以5开头的行

C:\Users\PXPXZ_~1\AppData\Local\Temp/ipykernel_9700/2926200544.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[condition]['expenses']=df['revenues']-df['profits']


In [14]:
# 查看是否修改成功
df[condition].head()
# 出现‘SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.Try using .loc[row_indexer,col_indexer] = value instead’时，有时修改会成功，有时不会成功。

,rank,name,revenues,revenue_percent_change,profits,profits_percent_change,assets,market_value,change_in_rank,employees
4,5,UnitedHealth Group,287597.0,11.80%,17285.0,12.20%,"$212,206","$479,830.30",-,"350,000"
49,50,MetLife,71080.0,4.80%,6554.0,21.20%,"$759,708","$57,986.50",-4,"43,000"
50,51,Prudential Financial,70934.0,24.40%,7724.0,-,"$937,582","$44,482.20",4,"40,916"
51,52,Albertsons,69690.4,11.60%,850.2,82.30%,"$26,598","$16,063.70",-,"300,000"
52,53,Walt Disney,67418.0,3.10%,1995.0,-,"$203,609","$249,718.10",-3,"171,000"


#### 2、原因
发出警告的代码df[condition]['expenses']=df['revenues']-df['profits']
相当于：df.get(condition).set(expenses),第一步的get发出了报警
##### 链式操作其实时两个步骤，先get后set，get得到的dataframe可能是view也可能是copy，pandas发出警告。
核心要诀：pandas的dataframe的修改写操作，只允许在源dataframe上进行，一步到位

#### 3、解决方法1
将get+set的两步操作，改成set的一步操作

In [15]:
df.loc[condition,'expense']=df['revenues']-df['profits']

In [16]:
df[condition].head()

,rank,name,revenues,revenue_percent_change,profits,profits_percent_change,assets,market_value,change_in_rank,employees,expense
4,5,UnitedHealth Group,287597.0,11.80%,17285.0,12.20%,"$212,206","$479,830.30",-,"350,000",270312.0
49,50,MetLife,71080.0,4.80%,6554.0,21.20%,"$759,708","$57,986.50",-4,"43,000",64526.0
50,51,Prudential Financial,70934.0,24.40%,7724.0,-,"$937,582","$44,482.20",4,"40,916",63210.0
51,52,Albertsons,69690.4,11.60%,850.2,82.30%,"$26,598","$16,063.70",-,"300,000",68840.2
52,53,Walt Disney,67418.0,3.10%,1995.0,-,"$203,609","$249,718.10",-3,"171,000",65423.0


In [ ]:
#### 4、解决方法2
如果需要预筛选数据做后续的处理分析，使用copy复制dataframe

In [17]:
df_5start=df[condition].copy()

In [18]:
df_5start.head()

,rank,name,revenues,revenue_percent_change,profits,profits_percent_change,assets,market_value,change_in_rank,employees,expense
4,5,UnitedHealth Group,287597.0,11.80%,17285.0,12.20%,"$212,206","$479,830.30",-,"350,000",270312.0
49,50,MetLife,71080.0,4.80%,6554.0,21.20%,"$759,708","$57,986.50",-4,"43,000",64526.0
50,51,Prudential Financial,70934.0,24.40%,7724.0,-,"$937,582","$44,482.20",4,"40,916",63210.0
51,52,Albertsons,69690.4,11.60%,850.2,82.30%,"$26,598","$16,063.70",-,"300,000",68840.2
52,53,Walt Disney,67418.0,3.10%,1995.0,-,"$203,609","$249,718.10",-3,"171,000",65423.0


In [19]:
df_5start['expense']=df['revenues']-df['profits']

In [20]:
df_5start.head()

,rank,name,revenues,revenue_percent_change,profits,profits_percent_change,assets,market_value,change_in_rank,employees,expense
4,5,UnitedHealth Group,287597.0,11.80%,17285.0,12.20%,"$212,206","$479,830.30",-,"350,000",270312.0
49,50,MetLife,71080.0,4.80%,6554.0,21.20%,"$759,708","$57,986.50",-4,"43,000",64526.0
50,51,Prudential Financial,70934.0,24.40%,7724.0,-,"$937,582","$44,482.20",4,"40,916",63210.0
51,52,Albertsons,69690.4,11.60%,850.2,82.30%,"$26,598","$16,063.70",-,"300,000",68840.2
52,53,Walt Disney,67418.0,3.10%,1995.0,-,"$203,609","$249,718.10",-3,"171,000",65423.0


#### 总之，pandas不允许先筛选子dataframe，再进行修改写入
要么使用loc实现一个步骤直接修改源dataframe
要么先复制一个子dataframe再一个步骤执行修改